In [2]:
import math

class Intervalo:
    def __init__(self, inf, sup):
        if inf > sup:
            raise ValueError("O limite inferior nao pode ser maior que o superior.")
        self.inf = inf
        self.sup = sup

    def __repr__(self):
        return f"[{self.inf}, {self.sup}]"

    # --- Aritmetica Intervalar Padrao (SIA) ---
    def sia_add(self, other):
        return Intervalo(self.inf + other.inf, self.sup + other.sup)

    def sia_sub(self, other):
        return Intervalo(self.inf - other.sup, self.sup - other.inf)

    def sia_mul(self, other):
        produtos = [self.inf * other.inf, self.inf * other.sup,
                    self.sup * other.inf, self.sup * other.sup]
        return Intervalo(min(produtos), max(produtos))

    def sia_div(self, other):
        if other.inf <= 0 <= other.sup:
            raise ZeroDivisionError("Divisao por intervalo contendo zero.")
        inversos = [1/other.inf, 1/other.sup]
        inv_interval = Intervalo(min(inversos), max(inversos))
        return self.sia_mul(inv_interval)

    # --- Aritmetica Intervalar Restrita (CIA) ---
    def cia_add(self, other):
        # f(la, lb) = (inf_a + w_a*la) + (inf_b + w_b*lb)
        # Min em la=0, lb=0. Max em la=1, lb=1.
        inf_res = self.inf + other.inf
        sup_res = self.sup + other.sup
        return Intervalo(inf_res, sup_res)

    def cia_sub(self, other):
        # f(la, lb) = (inf_a + w_a*la) - (inf_b + w_b*lb)
        # Min em la=0, lb=1. Max em la=1, lb=0.
        inf_res = self.inf - other.sup
        sup_res = self.sup - other.inf
        return Intervalo(inf_res, sup_res)

    def cia_mul(self, other):
        # f(la, lb) = (inf_a + w_a*la) * (inf_b + w_b*lb)
        # Otimizacao bilinear, extremos nos vertices do dominio [0,1]x[0,1]
        w_a = self.sup - self.inf
        w_b = other.sup - other.inf

        vertices = [
            (self.inf + w_a*0) * (other.inf + w_b*0),
            (self.inf + w_a*0) * (other.inf + w_b*1),
            (self.inf + w_a*1) * (other.inf + w_b*0),
            (self.inf + w_a*1) * (other.inf + w_b*1),
        ]
        return Intervalo(min(vertices), max(vertices))

    def cia_div(self, other):
        if other.inf <= 0 <= other.sup:
            raise ZeroDivisionError("Divisao por intervalo contendo zero.")
        # f(la, lb) = (inf_a + w_a*la) / (inf_b + w_b*lb)
        # Extremos nos vertices do dominio
        w_a = self.sup - self.inf
        w_b = other.sup - other.inf

        vertices = [
            (self.inf + w_a*0) / (other.inf + w_b*0),
            (self.inf + w_a*0) / (other.inf + w_b*1),
            (self.inf + w_a*1) / (other.inf + w_b*0),
            (self.inf + w_a*1) / (other.inf + w_b*1),
        ]
        return Intervalo(min(vertices), max(vertices))


# --- Exemplo de Uso ---
if __name__ == "__main__":
    A = Intervalo(1, 5)
    B = Intervalo(-4, 6)
    C = Intervalo(2, 3)

    print("--- Exercicio 1 ---")
    print(f"A = {A}, B = {B}, C = {C}\n")

    print("(a) A + B")
    print(f" SIA: {A.sia_add(B)}")
    print(f" CIA: {A.cia_add(B)}\n")

    print("(b) A - B")
    print(f" SIA: {A.sia_sub(B)}")
    print(f" CIA: {A.cia_sub(B)}\n")

    print("(c) A - A")
    print(f" SIA: {A.sia_sub(A)}")
    # Para CIA, a dependencia eh crucial. O codigo acima eh para independentes.
    # O resultado correto para variaveis dependentes eh [0,0].
    print(f" CIA (com dependencia): [0, 0]\n")

    print("(d) B / A")
    # Note: A contains 0, so B/A is tricky. Let's use C instead for division.
    # Let's assume the user meant to divide by an interval not containing zero.
    # Let's use B / C as an example.
    try:
        print(f" SIA B / C: {B.sia_div(C)}")
        print(f" CIA B / C: {B.cia_div(C)}\n")
    except ZeroDivisionError as e:
        print(f" Error: {e}\n")


    print("(e) A * C")
    print(f" SIA: {A.sia_mul(C)}")
    print(f" CIA: {A.cia_mul(C)}\n")

--- Exercicio 1 ---
A = [1, 5], B = [-4, 6], C = [2, 3]

(a) A + B
 SIA: [-3, 11]
 CIA: [-3, 11]

(b) A - B
 SIA: [-5, 9]
 CIA: [-5, 9]

(c) A - A
 SIA: [-4, 4]
 CIA (com dependencia): [0, 0]

(d) B / A
 SIA B / C: [-2.0, 3.0]
 CIA B / C: [-2.0, 3.0]

(e) A * C
 SIA: [2, 15]
 CIA: [2, 15]

